# Smart Recipe Analyzer with Pydantic AI #



In [9]:
import os
from pydantic import BaseModel, Field
from pydantic_ai import Agent
from typing import List, Optional
import asyncio
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()

True

In [2]:
class NutritionalInfo(BaseModel):
    """Nutritional information per serving"""
    calories: int = Field(description="Estimated calories per serving")
    protein_g: float = Field(description="Protein content in grams")
    carbs_g: float = Field(description="Carbohydrate content in grams")
    fat_g: float = Field(description="Fat content in grams")
    fiber_g: float = Field(description="Fiber content in grams")

In [3]:
class RecipeAnalysis(BaseModel):
    """Complete recipe analysis with structured data"""
    difficulty_level: str = Field(description="Easy, Medium, or Hard")
    prep_time_minutes: int = Field(description="Preparation time in minutes")
    cook_time_minutes: int = Field(description="Cooking time in minutes")
    servings: int = Field(description="Number of servings")
    cuisine_type: str = Field(description="Type of cuisine (e.g., Italian, Asian, etc.)")
    dietary_tags: List[str] = Field(description="Diet tags like vegetarian, gluten-free, etc.")
    nutrition: NutritionalInfo = Field(description="Nutritional information")
    missing_ingredients: List[str] = Field(description="Common ingredients that might be missing")
    cooking_tips: List[str] = Field(description="Helpful cooking tips for this recipe")
    healthiness_score: int = Field(description="Health score from 1-10 (10 being healthiest)")

In [10]:
def initialize_openai_client():
    """Initialize OpenAI client with API key from environment"""
    api_key = os.getenv('OPENAI_API_KEY')
    if not api_key:
        raise ValueError("OPENAI_API_KEY not found in environment variables. Please check your .env file.")
    
    return OpenAI(api_key=api_key)

In [11]:
openai_client = initialize_openai_client()

In [14]:
recipe_agent = Agent(
    model="gpt-3.5-turbo",
    result_type=RecipeAnalysis,
    system_prompt="""
    You are a professional chef and nutritionist. Analyze recipes and provide detailed, 
    accurate information about cooking difficulty, timing, nutrition, and helpful tips.

    Be realistic with your estimates and provide practical advice that home cooks can use.
    For nutritional information, provide reasonable estimates based on typical ingredient portions.
    """,
)

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_9176\4019619400.py:1: DeprecationWarning: `result_type` is deprecated, use `output_type` instead
  recipe_agent = Agent(


In [15]:
class RecipeAnalyzer:
    """Main class for analyzing recipes using Pydantic AI"""
    
    def __init__(self):
        self.agent = recipe_agent
    
    async def analyze_recipe(self, recipe_text: str) -> RecipeAnalysis:
        """
        Analyze a recipe and return structured data
        
        Args:
            recipe_text: The recipe text to analyze
            
        Returns:
            RecipeAnalysis: Structured analysis of the recipe
        """
        try:
            result = await self.agent.run(f"Analyze this recipe: {recipe_text}")
            return result.data
        except Exception as e:
            raise Exception(f"Error analyzing recipe: {str(e)}")
    
    def format_analysis(self, analysis: RecipeAnalysis) -> str:
        """Format the analysis for display"""
        return f"""
🍳 RECIPE ANALYSIS
==================

📊 Basic Info:
• Difficulty: {analysis.difficulty_level}
• Prep Time: {analysis.prep_time_minutes} minutes
• Cook Time: {analysis.cook_time_minutes} minutes
• Servings: {analysis.servings}
• Cuisine: {analysis.cuisine_type}
• Health Score: {analysis.healthiness_score}/10

🏷️ Dietary Tags: {', '.join(analysis.dietary_tags)}

📈 Nutrition (per serving):
• Calories: {analysis.nutrition.calories}
• Protein: {analysis.nutrition.protein_g}g
• Carbs: {analysis.nutrition.carbs_g}g
• Fat: {analysis.nutrition.fat_g}g
• Fiber: {analysis.nutrition.fiber_g}g

❓ Might be missing: {', '.join(analysis.missing_ingredients)}

💡 Cooking Tips:
{chr(10).join(f'• {tip}' for tip in analysis.cooking_tips)}
"""

In [16]:
async def main():
    # Sample recipe for demonstration
    sample_recipe = """
    Spaghetti Carbonara
    
    Ingredients:
    - 400g spaghetti
    - 200g pancetta or guanciale, diced
    - 4 large eggs
    - 100g Pecorino Romano cheese, grated
    - Black pepper
    - Salt
    
    Instructions:
    1. Cook spaghetti in salted boiling water until al dente
    2. Meanwhile, cook pancetta in a large pan until crispy
    3. In a bowl, whisk eggs with cheese and black pepper
    4. Drain pasta, reserving some pasta water
    5. Add hot pasta to pancetta pan, remove from heat
    6. Quickly stir in egg mixture, adding pasta water as needed
    7. Serve immediately with extra cheese and pepper
    """
    
    # Initialize analyzer
    analyzer = RecipeAnalyzer()
    
    print("🔍 Analyzing recipe...")
    
    try:
        # Analyze the recipe
        analysis = await analyzer.analyze_recipe(sample_recipe)
        
        # Display formatted results
        print(analyzer.format_analysis(analysis))
        
        # You can also access individual fields programmatically
        print(f"\n🎯 Quick Check:")
        print(f"This is a {analysis.difficulty_level.lower()} recipe")
        print(f"Total time: {analysis.prep_time_minutes + analysis.cook_time_minutes} minutes")
        print(f"Calories per serving: {analysis.nutrition.calories}")
        
    except Exception as e:
        print(f"❌ Error: {e}")

In [17]:
def recipe_to_shopping_list(analysis: RecipeAnalysis) -> List[str]:
    """Generate a shopping list from missing ingredients"""
    return [f"□ {ingredient}" for ingredient in analysis.missing_ingredients]

In [18]:
def is_healthy_recipe(analysis: RecipeAnalysis) -> bool:
    """Quick health check"""
    return analysis.healthiness_score >= 7

In [19]:
def get_time_category(analysis: RecipeAnalysis) -> str:
    """Categorize recipe by total time"""
    total_time = analysis.prep_time_minutes + analysis.cook_time_minutes
    if total_time <= 30:
        return "Quick meal"
    elif total_time <= 60:
        return "Regular meal"
    else:
        return "Special occasion"

In [23]:
await main()

🔍 Analyzing recipe...

🍳 RECIPE ANALYSIS

📊 Basic Info:
• Difficulty: Medium
• Prep Time: 15 minutes
• Cook Time: 15 minutes
• Servings: 4
• Cuisine: Italian
• Health Score: 8/10

🏷️ Dietary Tags: 

📈 Nutrition (per serving):
• Calories: 680
• Protein: 28.0g
• Carbs: 75.0g
• Fat: 30.0g
• Fiber: 4.0g

❓ Might be missing: 

💡 Cooking Tips:
• Ensure the pasta is al dente to prevent it from becoming mushy when mixed with the sauce.
• Gradually add the egg mixture to the pasta off the heat to avoid scrambling the eggs.
• Adjust the amount of black pepper to suit your taste preference.


🎯 Quick Check:
This is a medium recipe
Total time: 30 minutes
Calories per serving: 680


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_9176\2560727949.py:19: DeprecationWarning: `result.data` is deprecated, use `result.output` instead.
  return result.data
